<a href="https://colab.research.google.com/github/pafernannapi18/GithubIntro/blob/main/25th_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensamble Learning from Scratch

In [102]:
#Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

In [103]:
df = pd.read_csv("/content/drive/MyDrive/Train dataset/train.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [104]:
X = df[['GrLivArea', 'YearBuilt']]
X.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [105]:
y = df['SalePrice']
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

# Blending

In [106]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)
X_train.head()

,GrLivArea,YearBuilt
64,2034,1997
682,1291,1996
960,858,1958
1384,1258,1939
1100,438,1920


In [107]:
LM = linear_model.LinearRegression(normalize=True)
LM.fit(X_train, y_train)
LM_pred = LM.predict(X_test)
print("MSE(Linear Model):", mean_squared_error(y_test, LM_pred))

sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV = SVR(kernel='rbf')
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR):", mean_squared_error(y_test, SV_pred))

DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(Decision Tree Regressor):", mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = LM_pred*0.5+ SV_pred*0.1 +DR_pred*0.4
print("MSE(Mean):", mean_squared_error(y_test, mean_pred))

MSE(Linear Model): 2690647926.3776035
MSE(SVR): 7119069689.683064
MSE(Decision Tree Regressor): 2553443455.30812
MSE(Mean): 2470929971.5410113


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


In [108]:
Las = Lasso(alpha=0.1)
Las.fit(X_train, y_train)
Las_pred = Las.predict (X_test)
print("MSE(Lasso):", mean_squared_error(y_test, Las_pred))

sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV = SVR(kernel='rbf', C=0.5)
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR):", mean_squared_error(y_test, SV_pred))

DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(Decision Tree Regressor):", mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = LM_pred*0.5+ SV_pred*0.05 +DR_pred*0.45
print("MSE(Mean):", mean_squared_error(y_test, mean_pred))

MSE(Lasso): 2690647942.4335785
MSE(SVR): 7128920068.0851145
MSE(Decision Tree Regressor): 2553443455.30812
MSE(Mean): 2413207932.1617346


In [109]:
class Bagging():

  def fit(self, models, X, y):
    self.model_list =[]
    for j, model in enumerate(models):
      np.random.seed(j)
      self.rand_index = np.random.choice(X.index, X.shape[0], replace=True)
      self.X_rand = X.loc[self.rand_index]
    self.y_rand = y.loc[self.rand_index]
    self.model_list.append(model.fit(self.X_rand, self.y_rand))

  def predict(self, X, y):
    print(X.shape[0])
    print(len(self.model_list))
    self.pred_data = np.zeros((X.shape[0], len(self.model_list)))
    for i, model in enumerate(self.model_list):
      self.pred = model.predict(X)    
      self.pred_data[:, i] = self.pred
    self.final_pred = np.mean(self.pred_data, axis=1)
    print('MSE(mean): ', mean_squared_error(y, self.final_pred))
    return self.final_pred

We now compare the models alone verses using bagging and check which one is better.

In [110]:
np.random.seed(1)
rand_index = np.random.choice(X_train.index, 1022, replace=True)
np.random.seed(1)
rand_index = np.random.choice(X_train.index, X_train.shape[0], replace=True)
X_train1 = X_train.loc[rand_index]
y_train1 = y_train.loc[rand_index]

np.random.seed(2)
rand_index = np.random.choice(X_train.index, X_train.shape[0], replace=True)
X_train2 = X_train.loc[rand_index]
y_train2 = y_train.loc[rand_index]

np.random.seed(3)
rand_index = np.random.choice(X_train.index, X_train.shape[0], replace=True).tolist()
X_train3 = X_train.loc[rand_index]
y_train3 = y_train.loc[rand_index]

In [111]:
reg1 = GradientBoostingRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1 = reg1.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred1))
reg2 = GradientBoostingRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2 = reg2.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred2))
reg3 = GradientBoostingRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3 = reg3.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred3))
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print('MSE(mean): ', mean_squared_error(y_test, y_pred_mean))

MSE:  1954295273.3180141
MSE:  2246120995.481195
MSE:  1724088181.3387496
MSE(mean):  1658779275.8808985


In [112]:
reg1= xgb.XGBRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1 = reg1.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred1))
reg2 = xgb.XGBRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2 = reg2.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred2))
reg3 = xgb.XGBRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3 = reg3.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred3))
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print('MSE(mean): ', mean_squared_error(y_test, y_pred_mean))

[07:55:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE:  2213829666.0405197
[07:55:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE:  2247233980.532478
[07:55:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE:  1705477084.5991666
MSE(mean):  1687972552.6029377


In [113]:
reg1= lgb.LGBMRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1 = reg1.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred1))
reg2 = lgb.LGBMRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2 = reg2.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred2))
reg3 = lgb.LGBMRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3 = reg3.predict(X_test)
print('MSE: ', mean_squared_error(y_test, y_pred3))
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print('MSE(mean): ', mean_squared_error(y_test, y_pred_mean))
y_pred3[1]

MSE:  1996989149.286145
MSE:  2217785612.142733
MSE:  2337926346.668943
MSE(mean):  2010251814.6958816


149261.18481853406

# Using Bagging

In [114]:
models = [xgb.XGBRegressor(n_estimators=100, max_depth=5), GradientBoostingRegressor(n_estimators=100, max_depth=5), lgb.LGBMRegressor(n_estimators=100, max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)
bg.predict(X_test, y_test)[:10]

438
1
MSE(mean):  2074730307.085176


array([210257.87167613, 153809.26923892, 134665.09888615, 181467.21554413,
       130747.58722092,  88393.30294886, 199687.20209242, 128542.00157598,
       364138.4238314 , 148482.29049672])

# Stacking

In [115]:
class Stacking():

  def __init__(self, models, end_model):
    self.models = models
    self.end_model = end_model

  def fit(self, X,y,K,seed):
    self.K = K
    KF = KFold(n_splits=K, random_state=seed, shuffle=True)
    self.pred_data = np.array([])
    X = np.array(X)   
    y = np.array(y)
    self.preds = np.array([])
    self.model_list = []
    for i, model in enumerate(self.models):
      for train_index, test_index, in KF.split(X):
        self.model_list.append(model.fit(X[train_index], y[train_index]))
        y_pred = model.predict(X[test_index])
        self.preds = np.append(self.preds, y_pred)
    self.preds = self.preds.reshape(len(self.models), X.shape[0]).T
    self.end_model.fit(self.preds, y)

  def predict(self, X, y):
    X = np.array(X)   
    y = np.array(y)
    self.test_array = np.array([])
    self.final_test = np.zeros((X.shape[0], self.K))
    for model in self.model_list:
      y_pred_test = model.predict(X)
      self.test_array = np.append(self.test_array, y_pred_test)
    self.test_array = self.test_array.reshape(len(self.models)*self.K, X.shape[0]).T
    print("Test-->", self.test_array.shape)
    for j, i in enumerate(range(0, len(self.models)*self.K - self.K, self.K)):
      self.mean_pred = np.mean(self.test_array[:,i:i+self.K], axis=1)
      self.final_test[:,j] = self.mean_pred
    self.final_pred = self.end_model.predict(self.final_test)
    print("Last stage -->", self.final_test.shape)
    return self.final_pred

In [116]:
models = [xgb.XGBRegressor(n_estimators=100, max_depth=5), GradientBoostingRegressor(n_estimators=100, max_depth=5), lgb.LGBMRegressor(n_estimators=100, max_depth=5)]
St = Stacking(models, end_model=GradientBoostingRegressor(n_estimators=100, max_depth=5))
St.fit(X=X_train, y=y_train, K=3, seed=0)
y_pred_end = St.predict(X_test, y_test)
print("MSE(Stacking): ", mean_squared_error(y_test, y_pred_end)) 

[07:55:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Test--> (438, 9)
Last stage --> (438, 3)
MSE(Stacking):  15423360364.920626


Lets compare it to single models

In [117]:
X1_train, X2_train, X3_train = np.split(X_train,[340, 681])
y1_train, y2_train, y3_train = np.split(y_train,[340, 681])
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))
reg1_Gr = GradientBoostingRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_Gr = reg1_Gr.predict(X1_train)
print("MSE_1:", mean_squared_error(y1_train, y_pred1_reg1_Gr))

X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))
reg2_Gr = GradientBoostingRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_Gr = reg2_Gr.predict(X2_train)
print("MSE_2:", mean_squared_error(y2_train, y_pred2_reg2_Gr))

X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))
reg3_Gr = GradientBoostingRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_Gr = reg3_Gr.predict(X3_train)
print("MSE_2:", mean_squared_error(y3_train, y_pred3_reg3_Gr))
y_train_Gr = np.hstack((y_pred1_reg1_Gr, y_pred2_reg2_Gr, y_pred2_reg2_Gr))

MSE_1: 2073421958.2150207


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


MSE_2: 2342366773.4597898
MSE_2: 1444383801.9272323


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [118]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))
reg1_LG = lgb.LGBMRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_LG = reg1_LG.predict(X1_train)
print("MSE_1:", mean_squared_error(y1_train, y_pred1_reg1_LG))

X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))
reg2_LG = lgb.LGBMRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_LG = reg2_LG.predict(X2_train)
print("MSE_2:", mean_squared_error(y2_train, y_pred2_reg2_LG))

X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))
reg3_LG = lgb.LGBMRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_LG = reg3_LG.predict(X3_train)
print("MSE_3:", mean_squared_error(y3_train, y_pred3_reg3_LG))
y_train_LG = np.hstack((y_pred1_reg1_LG, y_pred2_reg2_LG, y_pred2_reg2_LG))


MSE_1: 1789291655.0779407
MSE_2: 2088962682.1700788
MSE_3: 1383718320.4924493


In [119]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))
reg1_RF = RandomForestRegressor(n_estimators=100, max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_RF = reg1_RF.predict(X1_train)
print("MSE_1:", mean_squared_error(y1_train, y_pred1_reg1_RF))

X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))
reg2_RF = RandomForestRegressor(n_estimators=500, max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_RF = reg2_RF.predict(X2_train)
print("MSE_2:", mean_squared_error(y2_train, y_pred2_reg2_RF))

X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))
reg3_RF = RandomForestRegressor(n_estimators=250, max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_RF = reg3_RF.predict(X3_train)
print("MSE_3:", mean_squared_error(y3_train, y_pred3_reg3_RF))
y_train_RF = np.hstack((y_pred1_reg1_RF, y_pred2_reg2_RF, y_pred2_reg2_RF))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


MSE_1: 1984794913.4124353


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


MSE_2: 2429725881.518442
MSE_3: 1179386321.594928


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [120]:
train_mix = np.vstack((y_train_Gr, y_train_LG, y_train_RF)).T

xgb.XGBRegressor
reg_end = xgb.XGBRegressor(n_estimators=250, max_depth=5).fit(train_mix, y_train)

X_test1_Gr = reg1_Gr.predict(X_test)
X_test2_Gr = reg2_Gr.predict(X_test)
X_test3_Gr = reg3_Gr.predict(X_test)
y_test_Gr = (X_test1_Gr + X_test2_Gr + X_test3_Gr)/3 

X_test1_LG = reg1_LG.predict(X_test)
X_test2_LG = reg2_LG.predict(X_test)
X_test3_LG = reg3_LG.predict(X_test)
y_test_LG = (X_test1_LG + X_test2_LG + X_test3_LG)/3

X_test1_RF = reg1_RF.predict(X_test)
X_test2_RF = reg2_RF.predict(X_test)
X_test3_RF = reg3_RF.predict(X_test)
y_test_RF = (X_test1_RF + X_test2_RF + X_test3_RF)/3

[07:55:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X h

In [121]:
X_test_end = np.vstack((y_test_Gr, y_test_LG, y_test_RF)).T
y_pred_end = reg_end.predict(X_test_end)

In [122]:
print("MSE(Stacking):", mean_squared_error(y_test, y_pred_end))

MSE(Stacking): 3042228510.1905713
